In [1]:
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
#from textblob import TextBlob
import pandas as pd
import numpy as np
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize


[nltk_data] Downloading package punkt to /Users/revan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/revan/nltk_data...


In [3]:
def SentimentalAnalysis(df, text_column):
    sentiment_analysis_columns = ['negativity', 'neutrality', 'positivity', 'compound']
    sid = SentimentIntensityAnalyzer()
    expanded_text_dataset = []
    column_names = list(df.columns.values)
    all_columns = column_names + sentiment_analysis_columns
    text_segment_list = list(df[text_column])
    for j in range(len(text_segment_list)):
        row_info = list(df.iloc[j])
        text = text_segment_list[j]
        ##Sentiment analysis
        try:
            ss = sid.polarity_scores(text)
            negativity = ss['neg']
            neutrality = ss['neu']
            positivity = ss['pos']
            compound = ss['compound']
            temp_data = row_info + [negativity, neutrality, positivity, compound]
            expanded_text_dataset.append(temp_data)
        except:
            print('Sentiment analysis not done: ' + str(j))
            temp_data = row_info + ['NA', 'NA', 'NA', 'NA']
            expanded_text_dataset.append(temp_data)
    ##Storing the results in a dataset
    sentiment_dataset_df = pd.DataFrame(expanded_text_dataset, columns = all_columns)
    #sentiment_dataset_df.to_csv('%s_sentiment_analysis.csv' %file_name)
    return sentiment_dataset_df

reviews_df = pd.read_csv("/Users/revan/Downloads/reviews.csv")

SentimentalAnalysis(reviews_df, 'comments')

Sentiment analysis not done: 1077
Sentiment analysis not done: 2786
Sentiment analysis not done: 7483
Sentiment analysis not done: 7827
Sentiment analysis not done: 8362
Sentiment analysis not done: 11415
Sentiment analysis not done: 11745
Sentiment analysis not done: 13838
Sentiment analysis not done: 14131
Sentiment analysis not done: 16185
Sentiment analysis not done: 18191
Sentiment analysis not done: 19098
Sentiment analysis not done: 19986
Sentiment analysis not done: 21122
Sentiment analysis not done: 21720
Sentiment analysis not done: 22215
Sentiment analysis not done: 25043
Sentiment analysis not done: 25446
Sentiment analysis not done: 26939
Sentiment analysis not done: 27128
Sentiment analysis not done: 28326
Sentiment analysis not done: 31537
Sentiment analysis not done: 32174
Sentiment analysis not done: 32398
Sentiment analysis not done: 35209
Sentiment analysis not done: 36419
Sentiment analysis not done: 38638
Sentiment analysis not done: 39919
Sentiment analysis not do

,listing_id,id,date,reviewer_id,reviewer_name,comments,negativity,neutrality,positivity,compound
0,1178162,4724140,2013-05-21,4298113,Olivier,My stay at islam's place was really cool! Good...,0.0,0.648,0.352,0.9626
1,1178162,4869189,2013-05-29,6452964,Charlotte,Great location for both airport and city - gre...,0.0,0.639,0.361,0.9061
2,1178162,5003196,2013-06-06,6449554,Sebastian,We really enjoyed our stay at Islams house. Fr...,0.0,0.767,0.233,0.9663
3,1178162,5150351,2013-06-15,2215611,Marine,The room was nice and clean and so were the co...,0.0,0.673,0.327,0.9267
4,1178162,5171140,2013-06-16,6848427,Andrew,Great location. Just 5 mins walk from the Airp...,0.0,0.637,0.363,0.8658
...,...,...,...,...,...,...,...,...,...,...
68270,7462268,80537457,2016-06-18,22034145,Antonio,Joe y su mujer son encantadores. La habitación...,0.0,0.946,0.054,0.34
68271,7462268,83640094,2016-07-03,40052513,Steve,Joe was on his way to Jamaica to be married! o...,0.014,0.822,0.164,0.9504
68272,7462268,85797088,2016-07-13,77129134,Nick,The room was very clean as were the bathrooms ...,0.0,0.784,0.216,0.9693
68273,7462268,97264637,2016-08-26,15799803,Vid,Staying in Lower Allston at Joe and Nancy's pl...,0.014,0.759,0.226,0.9957


In [16]:
def SentimentalAnalysis_v2(df, text_column):
    sentiment_analysis_columns = ['positivity_simple', 'negativity_simple']
    positive_words = pd.read_csv("/Users/revan/Downloads/positive_words.csv", header=None, names=['Words'])
    negative_words = pd.read_csv("/Users/revan/Downloads/negative_words.csv", encoding='ISO-8859-1', header=None, names=['Words'])    
    positive_set = set(positive_words['Words'])
    negative_set = set(negative_words['Words'])
    expanded_text_dataset = []
    column_names = list(df.columns.values)
    all_columns = column_names + sentiment_analysis_columns
    text_segment_list = list(df[text_column])
    for j in range(len(text_segment_list)):
        row_info = list(df.iloc[j])
        text = text_segment_list[j]
        words = str(text).split()                          
        ##Sentiment analysis
        try:            
            pos = [word for word in words if word in positive_set]
            neg = [word for word in words if word in negative_set]
            positive_simple = len(pos) / len(words)
            negative_simple = len(neg) / len(words)
            temp_data = row_info + [positive_simple, negative_simple]
            expanded_text_dataset.append(temp_data)
        except:
            print('Sentiment analysis not done: ' + str(j))
            temp_data = row_info + [None, None]
            expanded_text_dataset.append(temp_data)
    ##Storing the results in a dataset
    sentiment_dataset_df = pd.DataFrame(expanded_text_dataset, columns = all_columns)
    #sentiment_dataset_df.to_csv('%s_sentiment_analysis.csv' %file_name)
    return sentiment_dataset_df
 

In [17]:
print(SentimentalAnalysis_v2(reviews_df, 'comments'))

Sentiment analysis not done: 20327
       listing_id        id        date  reviewer_id reviewer_name  \
0         1178162   4724140  2013-05-21      4298113       Olivier   
1         1178162   4869189  2013-05-29      6452964     Charlotte   
2         1178162   5003196  2013-06-06      6449554     Sebastian   
3         1178162   5150351  2013-06-15      2215611        Marine   
4         1178162   5171140  2013-06-16      6848427        Andrew   
...           ...       ...         ...          ...           ...   
68270     7462268  80537457  2016-06-18     22034145       Antonio   
68271     7462268  83640094  2016-07-03     40052513         Steve   
68272     7462268  85797088  2016-07-13     77129134          Nick   
68273     7462268  97264637  2016-08-26     15799803           Vid   
68274     7462268  98550693  2016-08-31     90128094       Arianna   

                                                comments  positivity_simple  \
0      My stay at islam's place was really co

In [12]:
positive_words = pd.read_csv("/Users/revan/Downloads/positive_words.csv", header=None, names=['Words'])
positive_words.head(5)

,Words
0,a+
1,abound
2,abounds
3,abundance
4,abundant
